# Naive Bayes Classifier

###### Type of data we have
1. Class: no-recurrence-events, recurrence-events
2. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
3. menopause: lt40, ge40, premeno.
4. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
5. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
6. node-caps: yes, no.
7. deg-malig: 1, 2, 3.
8. breast: left, right.
9. breast-quad: left-up, left-low, right-up, right-low, central.
10. irradiat: yes, no.

In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv('dataset.csv')
dataset = np.array(data)

###### Removing datapoints with missing values
 * we remove occurances of the question mark symbol (?)
 * The Function return a data set with no datapoints containing missing values and a dataset only containing data with missing values


In [2]:
def deleteMissing(data):
    indices = np.array([])
    missingValues = np.empty((0,10))
    dataset = data
    for i in  range(len(dataset)):
        for j in range(len(dataset[i])):
            if dataset[i][j] == '?':
                indices = np.append(indices , [int(i)])
                missingValues = np.append(missingValues , np.array([dataset[int(i)]]), axis = 0 )

    for i in range(len(indices)):
        # We start deleting in from the dataset from the end of the indices array
        dataset = np.delete(dataset , int(indices[len(indices)-1-i]) , 0 )
    return (dataset , missingValues);



##### Setting appropriate datasets
* we start with a dataset the will have no occurences of data points with missing values
* Next is a dataset of all datapoints with missing values
* Finally we set set make a data set similar to the dataset withouth missing values which we will later change

In [3]:
dataset , dataWithMissingValues = deleteMissing(dataset)
datasetNMV = dataset # We later make this dataset a complete dataset with no occurences of missing data points

###### Below we split the data into training , testing and validating data 
* We first shuffle the data to randomise it
* We then split the data accordingly

In [4]:
# Give the data a random shuffle first
np.random.shuffle(dataset)

# Data split
trainingData , testData = dataset[:int(.70 * dataset.shape[0])] , dataset[int(.70 * dataset.shape[0]):]

print(trainingData.shape , testData.shape)

(193, 10) (84, 10)


##### The Priors
* The function below receives an array of classes
* It then returns an array of the amount of classes are there
* It also return an array of how many times each class appears in the input Array
* Lastly It returns an array of probabilities for each class
* This is to be used inside the Naive Bayes function

In [5]:
def Priors(classArray):
    probabilities = np.array([])
    classes , counts = np.unique(classArray , return_counts = True)
    
    for i in counts:
        probabilities = np.append(probabilities , [i/np.sum(counts)])
    
    return (classes , counts , probabilities)


##### Naive Bayes
* This class receives and index, idexing a feature to be used as an array of classes we are trying to classify
* It also receives a matric of training data
* it also receives a datapoint we are trying to classify
* `The Function returns an array of probabilities and an array of classes`

In [6]:
def NaiveBayes(classIndex , trainData , dataPoint):
    classes , classCounts , priorProbs = Priors(trainData[:,classIndex])
    totalProbs = np.array([])
    
    for i in range(len(classes)):
        
        prob = priorProbs[i]
        Class = classes[i]
        classOccurance = classCounts[i]
        
        for j in range(len(dataPoint)):
            
            if j == classIndex:
                continue
                
            fCount = 0
            
            for k in range(len(trainData)):
                if len(np.where(trainData[k][classIndex] == Class and trainData[k][j] == dataPoint[j])[0] > 0):
                    fCount += 1 
            
            fCount = fCount/classOccurance
            prob *= fCount 
            
        totalProbs = np.append(totalProbs , [prob])
    
    return (totalProbs , classes);
   


##### Dealing with missing Values
* We now need to predict what the missing values are using our Naive bayes classifier
* Once we have done that we need to add those data points back to the main data set
* Then lastly we need to run our classifier on the complete data set

In [7]:
# remember we have a matric of missing data points named dataWithMissingvalues

def fixMissingValues(data , missingData):
    dataset = data
    miss = missingData
    newData = np.empty((0,10))
    for dataPoint in miss:
        d = dataPoint      ## This was for testing, it works just fine so no need to change it
        if dataPoint[5] == '?':
            probs , classes = NaiveBayes(5 , dataset , dataPoint)
            d[5] = classes[np.argmax(probs)]
            dataset = np.append(dataset , [d] , axis = 0)
        elif dataPoint[8] == '?':
            probs , classes = NaiveBayes(8 , dataset , dataPoint)
            d[8] = classes[np.argmax(probs)]
            dataset = np.append(dataset , [d] , axis = 0)
      
    return dataset

### DatasetNMV is the Dataset with no occurences of missing values
* From this data we will create a training and a testing set
* Which we will later use for reporting accuracies

In [8]:
datasetNMV = fixMissingValues(datasetNMV, dataWithMissingValues)
np.random.shuffle(datasetNMV)
TrainSet , TestSet = datasetNMV[:int(.70*len(datasetNMV))] , datasetNMV[int(.70*len(datasetNMV)):]


 ### Confusion Matrix
* Below is a generalized confusion matrix taking in the training set and testing set
* The Function returns a confusion matric which we will use to analyse and deduce information for our model
* We are going to run the Model on the complete data set as well as the set with no missing value occurances
> `Information :` 
 * index is the feature index
 * TP is For TRUE POSITIVE
 * TN is TRUE NEGATIVE
 *  FN is FALSE NEGATIVE
 * FP is fALSE POSITIVE

In [9]:
def ConfusionMatric(trainingSet , testingSet):
    index = 0
    TP = 0 
    TN = 0 
    FN = 0 
    FP = 0
    
    for dataPoint in testingSet:
        probs , classes = NaiveBayes(index , trainingSet , dataPoint)
        predicted = classes[np.argmax(probs)]
        trueValue = dataPoint[index]
        
        if trueValue == 'recurrence-events':
            if predicted == trueValue:
                TP += 1
            else : 
                FN += 1
        else:
            if predicted == trueValue:
                TN += 1
            else : 
                FP += 1
    
    
    ConfusionMatrix = np.array([[TP , FP],[FN , TN]])
    return ConfusionMatrix


### Analysis
> Here we write a fucntion that receives a confusion matric this functions returns: 
* The Accuracy of the model
* The Specificity (AKA True Negative Rate)
* The Precision , Ration of correct Positive examples to Total positive examples
* The Recall, ratio of the number of correct positive examples out of those that were classified as positive

In [10]:
def Analysis(confusionMatric):
    Tp = confusionMatric[0][0]
    Fp = confusionMatric[0][1]
    Fn = confusionMatric[1][0]
    Tn = confusionMatric[1][1]
    
    accuracy = (Tp + Tn) / (Tp + Fp + Fn + Tn)
    specificity = Tn / (Tn + Fp)
    precision = Tp / (Tp + Fp)
    recall = Tp / (Tp + Fn)
    
    return(accuracy , specificity , precision , recall)

### Averages
* Below we run the algorithm again and again for a given dataset
* This function basically prints out stuff and returns nothing
* This function prints out the evarage amount Analysis of the date
* The function receives a dataset and the amount of times one would want the algorithm to run


In [11]:
def Averages(data , amount):
    dataset = data
    TP = 0 
    TN = 0 
    FN = 0 
    FP = 0
    
    for i in range(amount):
        np.random.shuffle(dataset)
        # Data split
        trainingData , testData = dataset[:int(.70 * dataset.shape[0])] , dataset[int(.70 * dataset.shape[0]):]
        
        confusionMatric = ConfusionMatric(trainingData, testData)
        TP += confusionMatric[0][0]
        FP += confusionMatric[0][1]
        FN += confusionMatric[1][0]
        TN += confusionMatric[1][1]
        
    ConfusionMatrix = np.array([[TP , FP],[FN , TN]])
    accuracy , specificity , precision , recall = Analysis(ConfusionMatrix)
    print("The average accuracy of the given dataset is: ", accuracy)
    print("The average recall of the given dataset is: ", recall)
    print("The average specificity of the given dataset is: " , specificity)
    print("The average precision of the given dataset is: " , precision)

## Finally Reporting Averages
* We have 2 datasets, one complete one and the other is one where the occurences of missing values have been removed
* Below we report averages of both these datasets
* We are going to start with the data with removed occurences then with the full data set
* This is run randomising test and training set 100, you can change this value in the input function

In [ ]:
Averages(dataset,100)
print()
print('----------------------------------------------------------------------------------------')
print()
Averages(datasetNMV,100)

The average accuracy of the given dataset is:  0.7288095238095238
The average recall of the given dataset is:  0.4765974765974766
The average specificity of the given dataset is:  0.8330809355544337
The average precision of the given dataset is:  0.5413777161349976

----------------------------------------------------------------------------------------

